In [7]:
import os, glob, cv2, numpy as np, nibabel as nib, matplotlib.pylab as plt
from sklearn.model_selection import train_test_split

In [8]:
master_path = r'./BraTS/'
folders = [folder for folder in os.listdir(os.path.join(master_path, 'BraTS2021_Training_Data')) if folder != '.DS_Store']

In [19]:
if not os.path.exists(os.path.join(master_path, 'BraTS2021_Training_Data_2D')): # BraTS2021_Training_Data
    os.makedirs(os.path.join(master_path, 'BraTS2021_Training_Data_2D'))
    for folder in folders:
        os.makedirs(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder))
        for img_type in ['flair', 'seg', 't1', 't1ce', 't2']:
            os.makedirs(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, img_type))

In [10]:
def CropAndResize(image):
    # Find the non-zero regions
    rows = np.any(image, axis=1)
    cols = np.any(image, axis=0)

    # Find the bounding box of the non-zero regions
    rows_indices = np.where(rows)[0]
    cols_indices = np.where(cols)[0]
    if len(rows_indices) != 0 or len(cols_indices) != 0:
        top_row = np.min(rows_indices)
        bottom_row = np.max(rows_indices)
        left_col = np.min(cols_indices)
        right_col = np.max(cols_indices)

        width = right_col - left_col
        height = bottom_row - top_row

        if width > height:
            top_row = top_row - (width - height) // 2
            bottom_row = bottom_row + (width - height) // 2
        else:
            left_col = left_col - (height - width) // 2
            right_col = right_col + (height - width) // 2

        # Crop the image
        cropped_image = image[top_row:bottom_row + 1, left_col:right_col + 1]
    else:
        cropped_image = image
    # Resize the image
    dim = [64,64]
    resized_image = cv2.resize(cropped_image, dim)
    return resized_image

def CropAndResizeWithRef(ref, image):
    # Find the non-zero regions
    rows = np.any(ref, axis=1)
    cols = np.any(ref, axis=0)

    # Find the bounding box of the non-zero regions
    rows_indices = np.where(rows)[0]
    cols_indices = np.where(cols)[0]
    if len(rows_indices) != 0 or len(cols_indices) != 0:
        top_row = np.min(rows_indices)
        bottom_row = np.max(rows_indices)
        left_col = np.min(cols_indices)
        right_col = np.max(cols_indices)

        width = right_col - left_col
        height = bottom_row - top_row

        if width > height:
            top_row = top_row - (width - height) // 2
            bottom_row = bottom_row + (width - height) // 2
        else:
            left_col = left_col - (height - width) // 2
            right_col = right_col + (height - width) // 2

        # Crop the image
        resized_ref = ref[top_row:bottom_row + 1, left_col:right_col + 1]
        cropped_image = image[top_row:bottom_row + 1, left_col:right_col + 1]
    else:
        resized_ref = ref
        cropped_image = image
    # Resize the image
    dim = [64,64]
    resized_image = cv2.resize(cropped_image, dim, interpolation = cv2.INTER_NEAREST)
    resized_ref = cv2.resize(resized_ref, dim)
    return resized_ref, resized_image.astype(np.uint8)

def Standardise(image) :
        if np.abs(image).sum() == 0:
            return image
        with np.errstate(divide='ignore',invalid='ignore'):
            image_nan = np.where(image == 0, np.nan, image)
            new_image = (image_nan - np.nanmean(image_nan)) / np.nanstd(image_nan)
            new_image = np.nan_to_num(new_image)
        return new_image

In [5]:
def convert(folders, org_folder = 'BraTS2021_Training_Data'):
    for counter, folder in enumerate(folders):
        img_path = os.path.join(master_path, org_folder, folder, folder + '_flair.nii.gz')
        lab_path = os.path.join(master_path, org_folder, folder, folder + '_seg.nii.gz')
        img = nib.load(img_path).get_fdata()
        lab = nib.load(lab_path).get_fdata()
        for i in range(img.shape[-1]):
            img_slice = img[:,:,i]
            lab_slice = lab[:,:,i]
            img_final, lab_final = CropAndResizeWithRef(img_slice, lab_slice)
            img_final = Standardise(img_final)
            # plt.imsave(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, 'flair', folder + '_flair_' + str(i+1) + '.png'), img_final)
            np.save(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, 'flair', folder + '_flair_' + str(i+1)), img_final)
            np.save(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, 'seg', folder + '_seg_' + str(i+1)), lab_final)
            # np.save(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, 'seg_bin', folder + '_seg_bin_' + str(i+1) + '.png'), (lab_final > 0).astype(np.uint8))
        for img_type in ['t1', 't1ce', 't2']:
            img_path = os.path.join(master_path, org_folder, folder, folder + '_' + img_type + '.nii.gz')
            img = nib.load(img_path).get_fdata()
            for i in range(img.shape[-1]):
                img_slice = img[:,:,i]
                img_final = Standardise(CropAndResize(img_slice))
                np.save(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, img_type, folder + '_' + img_type + '_' + str(i+1)), img_final)
                # plt.imsave(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, img_type, folder + '_' + img_type + '_' + str(i+1) + '.png'), img_final)
        print(f'Finished {counter+1}/{len(folders)}.', end="\r")


In [6]:
convert(folders)

In [11]:
# def convert_multi(folder):
#     org_folder = 'BraTS2021_Training_Data'
#     img_path = os.path.join(master_path, org_folder, folder, folder + '_flair.nii.gz')
#     lab_path = os.path.join(master_path, org_folder, folder, folder + '_seg.nii.gz')
#     img = nib.load(img_path).get_fdata()
#     lab = nib.load(lab_path).get_fdata()
#     for i in range(img.shape[-1]):
#         img_slice = img[:,:,i]
#         lab_slice = lab[:,:,i]
#         img_final, lab_final = CropAndResizeWithRef(img_slice, lab_slice)
#         img_final = Standardise(img_final)
#         # plt.imsave(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, 'flair', folder + '_flair_' + str(i+1) + '.png'), img_final)
#         np.save(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, 'flair', folder + '_flair_' + str(i+1)), img_final)
#         np.save(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, 'seg', folder + '_seg_' + str(i+1)), lab_final)
#         # np.save(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, 'seg_bin', folder + '_seg_bin_' + str(i+1) + '.png'), (lab_final > 0).astype(np.uint8))
#     for img_type in ['t1', 't1ce', 't2']:
#         img_path = os.path.join(master_path, org_folder, folder, folder + '_' + img_type + '.nii.gz')
#         img = nib.load(img_path).get_fdata()
#         for i in range(img.shape[-1]):
#             img_slice = img[:,:,i]
#             img_final = Standardise(CropAndResize(img_slice))
#             np.save(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, img_type, folder + '_' + img_type + '_' + str(i+1)), img_final)
#             # plt.imsave(os.path.join(master_path, 'BraTS2021_Training_Data_2D', folder, img_type, folder + '_' + img_type + '_' + str(i+1) + '.png'), img_final)

In [20]:
import os, multiprocessing
from convert_multi import convert_multi
if __name__ == "__main__":
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())  # Use all available CPU cores
    pool.map(convert_multi, folders)
    pool.close()
    pool.join()

# Train Val Test Split

In [ ]:
folders_train_tmp, folders_test = train_test_split(folders, test_size=0.1, random_state=42)
folders_train, folders_val = train_test_split(folders_train_tmp, test_size=0.22, random_state=42)

del(folders_train_tmp)

In [ ]:
# We want the file structure to be

# Main
# |-- Train
#     |-- flair_001.png
#     |-- flair_002.png
#     |-- .json
# |-- Val
#     |-- flair_001.png
#     |-- flair_002.png
#     |-- .json
# |-- Test
#     |-- flair_001.png
#     |-- flair_002.png
#     |-- .json

In [ ]:
if not os.path.exists(os.path.join(master_path, 'BraTS2021_Coco')): # BraTS2021_Training_Data
    os.makedirs(os.path.join(master_path, 'BraTS2021_Coco'))

datasets = ['Train', 'Val', 'Test']

def create_folder_structure(dataset, org_folder = 'BraTS2021_Training_Data'):
    dataset_ind = eval('folders_' + dataset.lower())
    for counter, folder in enumerate(dataset_ind):
        path_img = os.path.join(master_path, org_folder, folder, folder + '_flair.nii.gz')
        path_label = os.path.join(master_path, org_folder, folder, folder + '_seg.nii.gz')
        img = nib.load(path_img).get_fdata()
        label = nib.load(path_label).get_fdata()

        if not os.path.exists(os.path.join(master_path, 'BraTS2021_Coco', dataset)):
            os.makedirs(os.path.join(master_path, 'BraTS2021_Coco', dataset))
        # break images down
        for i in range(img.shape[-1]):
            img_slice = img[:,:,i]
            label_slice = label[:,:,i]

            # Separate labels
            label_slice1 = np.where(label_slice == 1, 1, 0)
            label_slice2 = np.where(label_slice == 2, 1, 0)
            label_slice4 = np.where(label_slice == 4, 1, 0)

            # Save images using matplotlib
            plt.imsave(os.path.join(master_path, 'BraTS2021_Coco', dataset, folder + '_flair' + '_' + str(i+1) + '.png'), img_slice)
            plt.imsave(os.path.join(master_path, 'BraTS2021_Coco', dataset, folder + '_seg' + '_level_1_' + str(i+1) + '.png'), label_slice1, cmap="gray")
            plt.imsave(os.path.join(master_path, 'BraTS2021_Coco', dataset, folder + '_seg' + '_level_2_' + str(i+1) + '.png'), label_slice2, cmap="gray")
            plt.imsave(os.path.join(master_path, 'BraTS2021_Coco', dataset, folder + '_seg' + '_level_4_' + str(i+1) + '.png'), label_slice4, cmap="gray")

            # Save images using cv2
            # np.save(os.path.join(master_path + '_array', folder, folder + '_flair' + '_' + str(i+1)), img_slice)
            # np.save(os.path.join(master_path + '_array', folder, folder + '_seg' + '_' + str(i+1)), label_slice)
        print(f'Finished {counter+1}/{len(dataset_ind)}: {folder} for {dataset} set.', end="\r")
    print('Finished all folders for ' + dataset + ' set. \n')


In [ ]:
for dataset in datasets:
    create_folder_structure(dataset)

In [ ]:
# To load an image
# np.load('/tmp/123.npy')

In [ ]:
for dataset in datasets:
    master_path = f"./BraTS/BraTS2021_Coco/{dataset}/"
    for level in ['level_1', 'level_2', 'level_4']:
        # find all level_1 files in Train etc
        mask_files = glob.glob(master_path + f'*{level}*.png')
        # Create file
        if not os.path.exists(os.path.join(master_path, level)):
            os.makedirs(os.path.join(master_path, level))
        # move all mask_files to it
        for mask_file in mask_files:
            os.rename(mask_file, os.path.join(master_path, level, os.path.basename(mask_file)))